In [1]:
%cd ..

/home/nikita/edu/ai-masters/nla1/project


In [7]:
from copy import deepcopy

import torch
from torch import nn
from torch.nn import functional as F

from conv_cp import CPConv2d

In [10]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.ReLU(),
    nn.Conv2d(16, 32, 3),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3),
    nn.Flatten()
)

cp_model = deepcopy(model)
for i, layer in enumerate(cp_model):
    if isinstance(layer, nn.Conv2d):
        cp_model[i] = CPConv2d(layer, 5)
        


In [11]:
x = torch.rand(32, 3, 32, 32)
y = model(x)
cp_y = cp_model(x)
print(F.mse_loss(y, cp_y))

tensor(0.0012, grad_fn=<MseLossBackward0>)


In [12]:
def get_parameter_count(model):
    return sum(p.numel() for p in model.parameters())

print("num of params orig model:", get_parameter_count(model))
print("num of params in cp model:", get_parameter_count(cp_model))

num of params orig model: 60512
num of params in cp model: 1751
